In [1]:
import pandas as pd
from pyspark.sql import SQLContext
from pyspark.sql.functions import *
from pyspark import SparkContext
sc = SparkContext()
sqlContext = SQLContext(sc)

In [2]:
import pandas as pd

In [3]:
# Reccomedation Inputs

Flight = ''
Price = True  # true for low, false to disable it.
Type = 'ECONOMY' # or  PROMO
Safety_Rating = '10.0'
Product_Rating = '8.0'
Flight_Time = True

In [4]:
rdd = sc.textFile('datasets/flights_info.csv')
rawdata = rdd.map(lambda x: (x.split(",")[6] ,x.split(",")[0], x.split(",")[1], int(x.split(",")[2]), float(x.split(",")[3]),x.split(",")[4],x.split(",")[5]))

rawdataframe = sqlContext.createDataFrame(rawdata, ['Airline',"Departure Time", "Arrival Time", "Flight Time(by mints)",'Price','Currency','Type'])
rawdataframe.show()

+----------+----------------+----------------+---------------------+------+--------+-------+
|   Airline|  Departure Time|    Arrival Time|Flight Time(by mints)| Price|Currency|   Type|
+----------+----------------+----------------+---------------------+------+--------+-------+
|   Pegasus|2020-06-06 06:20|2020-06-06 07:25|                   65|157.99|     TRY|ECONOMY|
|       THY|2020-06-06 11:15|2020-06-06 12:30|                   75|193.99|     TRY|ECONOMY|
|       THY|2020-06-06 16:30|2020-06-06 17:45|                   75|193.99|     TRY|ECONOMY|
|       THY|2020-06-06 19:25|2020-06-06 20:40|                   75|193.99|     TRY|ECONOMY|
|Anadolujet|2020-06-06 10:00|2020-06-06 11:05|                   65|201.99|     TRY|ECONOMY|
|   Pegasus|2020-06-06 20:25|2020-06-06 21:30|                   65|207.99|     TRY|ECONOMY|
|       THY|2020-06-06 19:30|2020-06-06 23:10|                  220|330.99|     TRY|ECONOMY|
|   Pegasus|2020-07-06 11:00|2020-07-06 12:10|                   70|15

In [5]:
rdd2 = sc.textFile('datasets/airlines_info.csv')

rdd3 = rdd2.map(lambda x: (x.split(",")[0], x.split(",")[1], x.split(",")[2], x.split(",")[3]))

rawdataframe2 = sqlContext.createDataFrame(rdd3, ["Airline", "Safety Rating", "Product Rating",'Jets'])
rawdataframe2.show()


+----------------+-------------+--------------+----------+
|         Airline|Safety Rating|Product Rating|      Jets|
+----------------+-------------+--------------+----------+
|      AnadoluJet|           10|           8.0|['737-800'|
|Turkish Airlines|           10|           8.5|['737-800'|
|         Pegasus|          8.5|           6.0|['737-800'|
+----------------+-------------+--------------+----------+



In [6]:

rawdataframe2 = rawdataframe.withColumn("Safety Rating",
       when(col("Airline") == "Anadolujet", "10.0")
      .when(col("Airline") == "THY", "10.0")
      .when(col("Airline") == "Pegasus", "8.5")
      .otherwise("null"))


rawdataframe3 = rawdataframe2.withColumn("Product Rating",
       when(col("Airline") == "Anadolujet", "7.5")
      .when(col("Airline") == "THY", "8.5")
      .when(col("Airline") == "Pegasus", "6.0")
      .otherwise("null"))




In [7]:
rawdataframe3.show()

+----------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|   Airline|  Departure Time|    Arrival Time|Flight Time(by mints)| Price|Currency|   Type|Safety Rating|Product Rating|
+----------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|   Pegasus|2020-06-06 06:20|2020-06-06 07:25|                   65|157.99|     TRY|ECONOMY|          8.5|           6.0|
|       THY|2020-06-06 11:15|2020-06-06 12:30|                   75|193.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-06-06 16:30|2020-06-06 17:45|                   75|193.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-06-06 19:25|2020-06-06 20:40|                   75|193.99|     TRY|ECONOMY|         10.0|           8.5|
|Anadolujet|2020-06-06 10:00|2020-06-06 11:05|                   65|201.99|     TRY|ECONOMY|         10.0|           7.5|
|   Pegasus|2020-06-06 2

In [8]:
# sorted by price 


dff2 = rawdataframe3.orderBy('Price', ascending=True)
dff2.show()


+-------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|Airline|  Departure Time|    Arrival Time|Flight Time(by mints)| Price|Currency|   Type|Safety Rating|Product Rating|
+-------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|Pegasus|2020-11-06 08:15|2020-11-06 09:25|                   70| 87.99|     TRY|  PROMO|          8.5|           6.0|
|Pegasus|2020-12-06 08:15|2020-12-06 09:25|                   70| 87.99|     TRY|  PROMO|          8.5|           6.0|
|Pegasus|2020-11-06 18:10|2020-11-06 19:20|                   70| 87.99|     TRY|  PROMO|          8.5|           6.0|
|Pegasus|2020-12-06 09:40|2020-12-06 10:50|                   70| 87.99|     TRY|  PROMO|          8.5|           6.0|
|Pegasus|2020-12-06 11:20|2020-12-06 12:30|                   70| 87.99|     TRY|  PROMO|          8.5|           6.0|
|Pegasus|2020-11-06 18:20|2020-11-06 19:30|     

In [9]:
print('Cheapest Price per Airline: ')
dff2.groupby('Airline').agg({'Price': 'min'}).show()

print('Highest Price per Airline: ')
dff2.groupby('Airline').agg({'Price': 'Max'}).show()

Cheapest Price per Airline: 
+----------+----------+
|   Airline|min(Price)|
+----------+----------+
|       THY|    113.99|
|Anadolujet|    160.99|
|   Pegasus|     87.99|
+----------+----------+

Highest Price per Airline: 
+----------+----------+
|   Airline|max(Price)|
+----------+----------+
|       THY|    726.99|
|Anadolujet|    711.99|
|   Pegasus|    339.99|
+----------+----------+



In [10]:
print('Shows the flights for alirlines with 10 safety score')

dff2.filter(dff2['Safety Rating'] == '10.0').show(5)

Shows the flights for alirlines with 10 safety score
+----------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|   Airline|  Departure Time|    Arrival Time|Flight Time(by mints)| Price|Currency|   Type|Safety Rating|Product Rating|
+----------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|       THY|2020-10-06 11:30|2020-10-06 12:35|                   65|113.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-10-06 07:15|2020-10-06 08:25|                   70|113.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-10-06 19:35|2020-10-06 20:40|                   65|113.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-10-06 15:45|2020-10-06 16:45|                   60|113.99|     TRY|ECONOMY|         10.0|           8.5|
|Anadolujet|2020-11-06 15:30|2020-11-06 16:30|                   60|160.99|     TRY|ECONOMY| 

In [11]:
print('Shows Airlines Safety Ratings: ')
dff2.groupby('Airline').agg({'Safety Rating': 'min'}).show()

print('Shows the lowest price of flights for airlines with 10 safety score')
dff2.filter(dff2['Safety Rating'] == '10.0').groupby('Airline').agg({'Price': 'min'}).show()

print('Shows the highest price of flights for airlines with 10 safety score')
dff2.filter(dff2['Safety Rating'] == '10.0').groupby('Airline').agg({'Price': 'max'}).show()


Shows Airlines Safety Ratings: 
+----------+------------------+
|   Airline|min(Safety Rating)|
+----------+------------------+
|       THY|              10.0|
|Anadolujet|              10.0|
|   Pegasus|               8.5|
+----------+------------------+

Shows the lowest price of flights for airlines with 10 safety score
+----------+----------+
|   Airline|min(Price)|
+----------+----------+
|       THY|    113.99|
|Anadolujet|    160.99|
+----------+----------+

Shows the highest price of flights for airlines with 10 safety score
+----------+----------+
|   Airline|max(Price)|
+----------+----------+
|       THY|    726.99|
|Anadolujet|    711.99|
+----------+----------+



In [12]:
print('Shows the flights for alirlines with 7 product score')

dff2.filter(dff2['Product Rating'] > '7.0').show(5)

Shows the flights for alirlines with 7 product score
+----------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|   Airline|  Departure Time|    Arrival Time|Flight Time(by mints)| Price|Currency|   Type|Safety Rating|Product Rating|
+----------+----------------+----------------+---------------------+------+--------+-------+-------------+--------------+
|       THY|2020-10-06 11:30|2020-10-06 12:35|                   65|113.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-10-06 07:15|2020-10-06 08:25|                   70|113.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-10-06 19:35|2020-10-06 20:40|                   65|113.99|     TRY|ECONOMY|         10.0|           8.5|
|       THY|2020-10-06 15:45|2020-10-06 16:45|                   60|113.99|     TRY|ECONOMY|         10.0|           8.5|
|Anadolujet|2020-11-06 15:30|2020-11-06 16:30|                   60|160.99|     TRY|ECONOMY| 

In [13]:
print('Shows Airlines Product Ratings: ')
dff2.groupby('Airline').agg({'Product Rating': 'min'}).show()

print('Shows the lowest price of flights for alirlines with 7 product score')
dff2.filter(dff2['Product Rating'] > '7.0').groupby('Airline').agg({'Price': 'min'}).show()

print('Shows the highest price of flights for alirlines with 7 product score')
dff2.filter(dff2['Product Rating'] > '7.0').groupby('Airline').agg({'Price': 'max'}).show()


Shows Airlines Product Ratings: 
+----------+-------------------+
|   Airline|min(Product Rating)|
+----------+-------------------+
|       THY|                8.5|
|Anadolujet|                7.5|
|   Pegasus|                6.0|
+----------+-------------------+

Shows the lowest price of flights for alirlines with 7 product score
+----------+----------+
|   Airline|min(Price)|
+----------+----------+
|       THY|    113.99|
|Anadolujet|    160.99|
+----------+----------+

Shows the highest price of flights for alirlines with 7 product score
+----------+----------+
|   Airline|max(Price)|
+----------+----------+
|       THY|    726.99|
|Anadolujet|    711.99|
+----------+----------+



In [14]:
print('Shows the flights that has a promo')

dff2.filter(dff2.Type.contains('PROMO')).show(5)

Shows the flights that has a promo
+-------+----------------+----------------+---------------------+-----+--------+-----+-------------+--------------+
|Airline|  Departure Time|    Arrival Time|Flight Time(by mints)|Price|Currency| Type|Safety Rating|Product Rating|
+-------+----------------+----------------+---------------------+-----+--------+-----+-------------+--------------+
|Pegasus|2020-11-06 07:10|2020-11-06 08:20|                   70|87.99|     TRY|PROMO|          8.5|           6.0|
|Pegasus|2020-11-06 08:15|2020-11-06 09:25|                   70|87.99|     TRY|PROMO|          8.5|           6.0|
|Pegasus|2020-11-06 18:10|2020-11-06 19:20|                   70|87.99|     TRY|PROMO|          8.5|           6.0|
|Pegasus|2020-11-06 18:20|2020-11-06 19:30|                   70|87.99|     TRY|PROMO|          8.5|           6.0|
|Pegasus|2020-12-06 06:10|2020-12-06 07:20|                   70|87.99|     TRY|PROMO|          8.5|           6.0|
+-------+----------------+-----------

In [15]:
print('Shows Airlines min price per type: ')
dff2.groupby('Airline','Type').agg({'Price': 'min'}).show()

print('Shows Airlines max price per type: ')
dff2.groupby('Airline','Type').agg({'Price': 'max'}).show()



Shows Airlines min price per type: 
+----------+-------+----------+
|   Airline|   Type|min(Price)|
+----------+-------+----------+
|   Pegasus|ECONOMY|    107.99|
|   Pegasus|  PROMO|     87.99|
|       THY|ECONOMY|    113.99|
|Anadolujet|ECONOMY|    160.99|
+----------+-------+----------+

Shows Airlines max price per type: 
+----------+-------+----------+
|   Airline|   Type|max(Price)|
+----------+-------+----------+
|   Pegasus|ECONOMY|    339.99|
|   Pegasus|  PROMO|     97.99|
|       THY|ECONOMY|    726.99|
|Anadolujet|ECONOMY|    711.99|
+----------+-------+----------+



In [16]:
print('Shows Airlines min Fly Time: ')
dff2.groupby('Airline').agg({'Flight Time(by mints)': 'min'}).show()

print('Shows Airlines max Fly Time: ')
dff2.groupby('Airline').agg({'Flight Time(by mints)': 'max'}).show()



Shows Airlines min Fly Time: 
+----------+--------------------------+
|   Airline|min(Flight Time(by mints))|
+----------+--------------------------+
|       THY|                        60|
|Anadolujet|                        60|
|   Pegasus|                        65|
+----------+--------------------------+

Shows Airlines max Fly Time: 
+----------+--------------------------+
|   Airline|max(Flight Time(by mints))|
+----------+--------------------------+
|       THY|                       240|
|Anadolujet|                       245|
|   Pegasus|                        70|
+----------+--------------------------+



In [17]:
# rawdataframe3.toPandas().to_csv('datasets/Flights_Airlines_Data_Combined.csv')